# Sudoku Solver

### Steps
1. Script ingests gameboard as a list of lists
2. 'Dumb' algorithms to check
    1. Row
    2. Column
    3. own square
3. Stores possibilities in 'possibilities' list
4. Solving loop
    1. try random possibility from upper left
    2. if no numbers in possibility, up one loop, backtrack
    4. update possibilities
    3. try next number

In [1]:
import sys
import copy
import random

In [104]:
def reset_board(board_name):
    if board_name == "board1":
        board = board1
    elif board_name == "board2":
        board = board2
    elif board_name == "board3":
        board = board3
    elif board_name == "board4":
        board = board4
    return board

In [75]:
board1 = [
    [ 3 , '~', '~', '~',  1 ,  9 , '~', '~', '~'],
    ['~',  9 ,  6 , '~',  2 ,  8 ,  4 ,  3 ,  1 ],
    [ 1 ,  5 ,  2 , '~',  4 ,  7 ,  6 ,  8 , '~'],
    ['~',  6 ,  7 , '~',  8 , '~', '~', '~', '~'],
    [ 8 , '~',  5 , '~', '~', '~',  3 , '~',  6 ],
    ['~', '~', '~',  1 , '~', '~', '~',  7 , '~'],
    ['~', '~', '~',  2 ,  3 , '~',  9 ,  6 , '~'],
    [ 5 , '~', '~', '~',  6 , '~', '~', '~', '~'],
    ['~',  2 , '~',  8 , '~', '~',  1 , '~',  7 ],
]

In [125]:
board2 = [
    ['~', '~', '~',  1 , '~',  5 , '~',  6 ,  8 ],
    ['~', '~', '~', '~', '~', '~',  7 , '~',  1 ],
    [ 9 , '~',  1 , '~', '~', '~', '~',  3 , '~'],
    ['~', '~',  7 , '~',  2 ,  6 , '~', '~', '~'],
    [ 5 , '~', '~', '~', '~', '~', '~', '~',  3 ],
    ['~', '~', '~',  8 ,  7 , '~',  4 , '~', '~'],
    ['~',  3 , '~', '~', '~', '~',  8 , '~',  5 ],
    [ 1 , '~',  5 , '~', '~', '~', '~', '~', '~'],
    [ 7 ,  9 , '~',  4 , '~',  1 , '~', '~', '~'],
]

In [121]:
board3 = [
    [ 3 , '~',  5 , '~', '~', '~',  2 , '~', '~'],
    ['~', '~',  9 ,  2 , '~', '~',  1 ,  7 , '~'],
    [ 2 , '~', '~', '~', '~',  9 , '~',  8 ,  3 ],
    ['~', '~', '~',  4 ,  7 , '~',  3 , '~', '~'],
    ['~',  1 , '~', '~', '~', '~', '~',  5 , '~'],
    ['~', '~',  2 , '~',  5 ,  1 , '~', '~', '~'],
    [ 5 ,  2 , '~',  1 , '~', '~', '~', '~',  8 ],
    ['~',  9 ,  4 , '~', '~',  8 ,  7 , '~', '~'],
    ['~', '~',  7 , '~', '~', '~',  4 , '~',  5 ],
]

In [115]:
board4 = [
    [ 5 ,  3 , '~', '~',  7 , '~', '~', '~', '~'],
    [ 6 , '~', '~',  1 ,  9 ,  5 , '~', '~', '~'],
    ['~',  9 ,  8 , '~', '~', '~', '~',  6 , '~'],
    [ 8 , '~', '~', '~',  6 , '~', '~', '~',  3 ],
    [ 4 , '~', '~',  8 , '~',  3 , '~', '~',  1 ],
    [ 7 , '~', '~', '~',  2 , '~', '~', '~',  6 ],
    ['~',  6 , '~', '~', '~', '~',  2 ,  8 , '~'],
    ['~', '~', '~',  4 ,  1 ,  9 , '~', '~',  5 ],
    ['~', '~', '~', '~',  8 , '~', '~',  7 ,  9 ],
]

In [104]:
board = [
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
    ['~', '~', '~', '~', '~', '~', '~', '~', '~'],
]

In [19]:
def display_game_board(board):
    board_array = [entry for row in board for entry in row]
    for i, value in enumerate(board_array):
        if i == 0: sys.stdout.write('---' * 9 + '----\n|')
        else: 
            if i % 9 == 0: sys.stdout.write('|\n')
            if i % 27 == 0: sys.stdout.write('---' * 9 + '----\n')
            if i % 3 == 0: sys.stdout.write('|')
        sys.stdout.write(f' {value} ')
        if i == 80:
            sys.stdout.write('|\n' + '---' * 9 + '----')

In [6]:
def check_row(board, row):
    values_in_row = []
    for value in board[row]:
        if isinstance(value, int): values_in_row.append(value)
    return values_in_row

In [7]:
def check_column(board, column):
    values_in_column = []
    for i in range(9):
        if isinstance(board[i][column], int): values_in_column.append(board[i][column])
    return values_in_column

In [8]:
def check_square(board, row, column):
    values_in_square = []

    if 0 <= row <= 2:
        rows_to_check = [0, 1, 2]
    elif 3 <= row <= 5:
        rows_to_check = [3, 4, 5]
    elif 6 <= row <= 8:
        rows_to_check = [6, 7, 8]

    if 0 <= column <= 2:
        columns_to_check = [0, 1, 2]
    elif 3 <= column <= 5:
        columns_to_check = [3, 4, 5]
    elif 6 <= column <= 8:
        columns_to_check = [6, 7, 8]

    for row in rows_to_check:
        for column in columns_to_check:
            if isinstance(board[row][column], int):
                values_in_square.append(board[row][column])
    
    return values_in_square
    

In [9]:
def find_value_options(board, row, column):
    default_options = set(x for x in range(1, 10))

    values_in_row = check_row(board, row)
    values_in_column = check_column(board, column)
    values_in_square = check_square(board, row, column)

    values_taken = set(values_in_row + values_in_column + values_in_square)

    value_options = list(default_options - values_taken)

    return(value_options)

In [58]:
# Find values guarenteed by row, column, and square
# Adds them to board
def write_guarenteed_values(board):
    # print('Original Game Board')
    # display_game_board(board)
    # print('\nGuarenteed Spots Filled In')
    new_board = copy.deepcopy(board)
    x = 1
    while True:
        updated_board = False
        for row in range(9):
            for column in range(9):
                if isinstance(new_board[row][column], int):
                    continue
                value_options = find_value_options(new_board, row, column)
                if len(value_options) == 1:
                    new_board[row][column] = value_options[0]
                    updated_board = True
        if updated_board == False:
            break
    # display_game_board(new_board)
    return new_board

In [14]:
def is_board_solved(board):
    for row in range(9):
        for column in range(9):
            if isinstance(board[row][column], str):
                return False
    return True

In [15]:
def find_missing_values(board):
    missing_values = []
    for row in range(9):
        for column in range(9):
            if isinstance(board[row][column], str):
                missing_values.append((row,column))
    return missing_values                

In [152]:
def recursive_solve(board, recursion_count=None, index=None, missing_values=None, values_tried=None):
    print()
    if recursion_count == None:
        recursion_count = 0
    if index == None:
        index = 0
    if missing_values == None:
        missing_values = find_missing_values(board)
    if values_tried == None:
        values_tried = [[] for x in range(len(missing_values))]
    if is_board_solved(board):
        return

    row, column = missing_values[index]
    board[row][column] = '~'
    value_options = find_value_options(board, row, column)
    value_options = list(set(value_options) - set(values_tried[index]))
    value_options.sort()
    print(f'this board has been tried {recursion_count} times recursively')
    print(f'row = {row}, column = {column}')
    print(f'value options = {value_options}')
    print(f'already_tried {values_tried[index]}')
    print(f'index = {index}')
    print(f'values_tried = {values_tried}')
    display_game_board(board)
    if len(value_options) == 0:
        recursive_solve(board, recursion_count + 1, index - 1, missing_values, values_tried)
    else:
        value_to_try = value_options[0]
        print(f'\nvalue to try = {value_to_try}')
        values_tried[index].append(value_to_try)
        for backtrack_index in range(index + 1, len(values_tried)):
            values_tried[backtrack_index] = []
        board[row][column] = value_to_try
        recursive_solve(board, recursion_count + 1,index + 1, missing_values, values_tried)


In [120]:
sys.setrecursionlimit(10000)
print(sys.getrecursionlimit())

10000


In [165]:
board = reset_board("board3")

In [166]:
display_game_board(board)

-------------------------------
| 3  ~  5 | ~  ~  ~ | 2  ~  ~ |
| ~  ~  9 | 2  ~  ~ | 1  7  ~ |
| 2  ~  ~ | ~  ~  9 | ~  8  3 |
-------------------------------
| ~  ~  ~ | 4  7  ~ | 3  ~  ~ |
| ~  1  ~ | ~  ~  ~ | ~  5  ~ |
| ~  ~  2 | ~  5  1 | ~  ~  ~ |
-------------------------------
| 5  2  ~ | 1  ~  ~ | ~  ~  8 |
| ~  9  4 | ~  ~  8 | 7  ~  ~ |
| ~  ~  7 | ~  ~  ~ | 4  ~  5 |
-------------------------------

In [167]:
default_stdout = sys.stdout
board_to_solve = write_guarenteed_values(board)
# board_to_solve = board
try:
    sys.stdout = open('output.txt', 'wt')
    recursive_solve(board_to_solve)
except:
    RecursionError
    sys.stdout = default_stdout
    print(f'MAX RECURSION DEPTH OF {sys.getrecursionlimit()} MET')
sys.stdout = default_stdout
print(f'Is board solved? {is_board_solved(board_to_solve)}')
display_game_board(board_to_solve)

Is board solved? True
-------------------------------
| 3  7  5 | 8  1  4 | 2  6  9 |
| 8  6  9 | 2  3  5 | 1  7  4 |
| 2  4  1 | 7  6  9 | 5  8  3 |
-------------------------------
| 9  5  8 | 4  7  6 | 3  1  2 |
| 4  1  6 | 3  8  2 | 9  5  7 |
| 7  3  2 | 9  5  1 | 8  4  6 |
-------------------------------
| 5  2  3 | 1  4  7 | 6  9  8 |
| 6  9  4 | 5  2  8 | 7  3  1 |
| 1  8  7 | 6  9  3 | 4  2  5 |
-------------------------------